# Library

In [1]:
!pip install keras_ocr

     |████████████████████████████████| 166 kB 2.4 MB/s 
     |████████████████████████████████| 9.5 MB 9.4 MB/s 
     |████████████████████████████████| 803 kB 38.4 MB/s 
     |████████████████████████████████| 179 kB 45.1 MB/s 
     |████████████████████████████████| 131 kB 51.3 MB/s 
     |████████████████████████████████| 50 kB 5.8 MB/s 
  Created wheel for keras-ocr: filename=keras_ocr-0.8.4-py3-none-any.whl size=148638 sha256=e0cc18c2f351b802b7dd0662ae5a7d84861bc0c227777edc77d567f4326c9981
  Stored in directory: /root/.cache/pip/wheels/a6/d8/34/518e393455207f8752d5777d1c174617d200cab5a5ed7bf2ad
  Created wheel for validators: filename=validators-0.15.0-py3-none-any.whl size=18371 sha256=d3962880bbd90ece05a98aae5cd11707f82427c0c46dcfacfefbd3da9f979fe7
  Stored in directory: /root/.cache/pip/wheels/ff/a1/08/0650ea8df7821ecd87760d16213b53dd23403927a69a388268
Successfully built keras-ocr validators


In [2]:
!pip freeze > requirements.txt

In [3]:
import matplotlib.pyplot as plt
import keras_ocr

import numpy as np
import pandas as pd

from time import time
import re
import itertools
import multiprocessing
import gc

In [4]:
print('Numpy version:', np.__version__)
print('Pandas version:', pd.__version__)

Numpy version: 1.18.1
Pandas version: 1.0.3


# Dataset

In [5]:
!ls /kaggle/input

shopee-product-detection-student


In [6]:
# df = pd.read_csv('/kaggle/input/shopee-product-detection-student/train.csv', dtype='object')
df = pd.read_csv('/kaggle/input/shopee-product-detection-student/test.csv', dtype='object')

df['category'] = df['category'].apply(lambda c: str(c).zfill(2))
df.head()

,filename,category
0,fd663cf2b6e1d7b02938c6aaae0a32d2.jpg,43
1,c7fd77508a8c355eaab0d4e10efd6b15.jpg,43
2,127f3e6d6e3491b2459812353f33a913.jpg,43
3,5ca4f2da11eda083064e6c36f37eeb81.jpg,43
4,46d681a542f2c71be017eef6aae23313.jpg,43


In [7]:
paths = []
for i in df.index:
#     paths.append(f'/kaggle/input/shopee-product-detection-student/train/train/train/{df.loc[i, "category"]}/{df.loc[i, "filename"]}')
    paths.append(f'/kaggle/input/shopee-product-detection-student/test/test/test/{df.loc[i, "filename"]}')

# Config OCR

In [8]:
pipeline = keras_ocr.pipeline.Pipeline()

Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5


In [9]:
# part 1 - 4 : train image
# part 1 = 0:26347
# part 2 = 26347:52695
# part 3 = 52695:79042
# part 4 = 79042:TOTAL_IMAGES
# part 5 : test image
# part 5 = 0:TOTAL_IMAGES (CURRENT)
TOTAL_IMAGES = len(paths)
BATCH_PREDICT = 5

START_INDEX = 0
END_INDEX = TOTAL_IMAGES

list_texts = []

# Predict

In [10]:
for i in range(START_INDEX, END_INDEX, BATCH_PREDICT):
    try:
        if i + BATCH_PREDICT < END_INDEX:
            images = [keras_ocr.tools.read(p) for p in paths[i:i + BATCH_PREDICT]]
        else:
            images = [keras_ocr.tools.read(p) for p in paths[i:END_INDEX]]

        prediction_groups = pipeline.recognize(images)

        for x in range(len(prediction_groups)):
            texts = []

            for y in range(len(prediction_groups[x])):
                texts.append(prediction_groups[x][y][0])

            list_texts.append(texts)
        gc.collect()
    except:
        if i + BATCH_PREDICT < END_INDEX:
            for j in range(BATCH_PREDICT):
                list_texts.append([])
        else:
            for j in range(END_INDEX - i):
                list_texts.append([])

In [11]:
sr_text = pd.Series(list_texts)
sr_text

0             [kafe, murah, ikios, meja, wao8216316690, e]
1                              [comi, pusat, iniatur, wwn]
2                         [r, r, girl, cid, cocoo, shopee]
3                  [pee, sniper, bicmcle, helmets, ntilos]
4                                [1121, 10, 2, 9, 3, 5166]
                               ...                        
12181                                                   []
12182                                             [sanisk]
12183                    [happypumping, l, happy, pumping]
12184    [hot, 1004, money, back, frey, boas, estines, ...
12185                                                   []
Length: 12186, dtype: object

In [12]:
sr_text.to_csv('test2.csv', index=False, header=False)